In [ ]:
import shutil
import os
import numpy as np
import matplotlib.pylab as plt
import tensorflow as tf

import sys
sys.path.append(os.path.abspath("../framework"))
sys.path.append(os.path.abspath("../concrete"))

In [ ]:
from AsmAction import AsmAction
from AsmSimulator import AsmSimulator
from ConcObservation import ConcObservation
from ConcAgentFactory import ConcAgentFactory
from ConcBuildOrder import ConcBuildOrder
from ConcEnvrionmentFactory import ConcEnvironmentFactory
from ConcRewardGiverFactory import ConcRewardGiverFactory
from ConcTrainerFactory import ConcTrainerFactory
from ConcValueFunctionApproximatorFactory import ConcValueFunctionApproximatorFactory
from framework import Trainer, ObservationSequence

## S100: Check the responce of Asm Simulator to the actions

### S110: steady state with various set values of DO

In [ ]:
def checkSteadyState(action):
    # ignore the fluctuation of inflow
    asmSimulator =  AsmSimulator(h = 1.0)
    asmSimulator.init()

    Y = []
    X = []

    for _ in range(14): # 1[step] = [1day]
        asmSimulator.update(action)
        observation = asmSimulator.observe()
        Y.append(observation.getValue())
        X.append(asmSimulator.x)

    Ynumpy = np.concatenate(Y, axis=0) # (*, nPv = 1)
    Xnumpy = np.stack(X, axis=0) # (*, nAsm)

    plt.figure()
    plt.plot(Ynumpy)
    plt.title('NH4')

    for k1, name in enumerate(asmSimulator.asmVarNames):
        print("{0:5s} {1:8.2f}".format(name, asmSimulator.x[k1]))

#### DO = 3.0

In [ ]:
action = AsmAction(np.ones((1,1)) * 10.0) # (1, nMv = 1)
Do = action.getActionOnEnvironment()
checkSteadyState(action)

#### DO = 0.8

In [ ]:
action = AsmAction(np.ones((1,1)) * -1.0) # (1, nMv = 1)
Do = action.getActionOnEnvironment()
checkSteadyState(action)

#### DO = 0.14

In [ ]:
action = AsmAction(np.ones((1,1)) * -3.0) # (1, nMv = 1)
Do = action.getActionOnEnvironment()
checkSteadyState(action)

### S120: steady state with cyclic pertubated inflow

In [ ]:
def checkSteadyStateWithPertubation(action):
    # ignore the fluctuation of inflow
    asmSimulator =  AsmSimulator(amplitudeDv=2.0)
    asmSimulator.init()

    Y = []
    X = []

    for _ in range(96*3): # 96[step] = [1day]
        asmSimulator.update(action)
        observation = asmSimulator.observe()
        Y.append(observation.getValue())
        X.append(asmSimulator.x)

    Ynumpy = np.concatenate(Y, axis=0) # (*, nPv = 1)
    Xnumpy = np.stack(X, axis=0) # (*, nAsm)

    plt.figure()
    plt.plot(Ynumpy)
    plt.title('NH4')

#### DO = 3.0

In [ ]:
action = AsmAction(np.ones((1,1)) * 10.0) # (1, nMv = 1)
Do = action.getActionOnEnvironment()
checkSteadyStateWithPertubation(action)

#### DO = 0.8

In [ ]:
action = AsmAction(np.ones((1,1)) * -1.0) # (1, nMv = 1)
Do = action.getActionOnEnvironment()
checkSteadyStateWithPertubation(action)

#### DO = 0.36

In [ ]:
action = AsmAction(np.ones((1,1)) * -2.0) # (1, nMv = 1)
Do = action.getActionOnEnvironment()
checkSteadyStateWithPertubation(action)